In [1]:
%matplotlib qt5

import os
import numpy as np
import SimpleITK as sitk
import matplotlib
import matplotlib.pyplot as plt
from copy import copy
from tqdm import trange

def plot_image(arr, title='', vmin=None, vmax=None):
    cm = copy(plt.cm.viridis)
    plt.figure(figsize=(20,20))
    im = plt.imshow(arr, vmin=vmin, vmax=vmax, cmap=cm, interpolation='none')
    plt.title(title)
    im.cmap.set_under('b')
    im.cmap.set_over('r')
    im.cmap.set_bad('pink')
    plt.colorbar()
    plt.show()

In [2]:
prefix = r'F:\Jasper\Data\20221014_HamNCheese_Best_of_the_best\results'

ff_filename = 'open_stack_th0.npy'
ff_file_path = os.path.join(prefix, ff_filename)
ff = np.load(ff_file_path)
print(ff.shape, type(ff), ff.dtype)

# proj_filename = 'projs_stack_th0.npy'
# proj_file_path = os.path.join(prefix, proj_filename)
# proj = np.load(proj_file_path)
# print(proj.shape, type(proj), proj.dtype)

(39, 10, 512, 512) <class 'numpy.ndarray'> float64


In [3]:
arr_3d = proj[0] / np.sum(ff, axis=1)[0]
arr = arr_3d[0]

In [4]:

# print(np.min(arr), np.max(arr), np.mean(arr), np.median(arr), np.std(arr))
# print(np.nanmin(arr), np.nanmax(arr), np.nanmean(arr), np.nanmedian(arr), np.nanstd(arr))

# plt.hist(arr.flatten(), bins=200)
# plt.yscale('log')
# plt.show()

# plt.hist(arr[0].flatten(), bins=200)
# plt.yscale('log')
# plt.show()

# plt.hist(arr[-1].flatten(), bins=200)
# plt.yscale('log')
# plt.show()

plt.close('all')

# diff_pixels = 512 - 1


b = np.diff(arr, axis=0)
c = np.diff(arr, axis=1)
# # b = np.diff(arr[0], axis=0)[:diff_pixels,:diff_pixels]
# # c = np.diff(arr[0], axis=1)[:diff_pixels,:diff_pixels]
# b = np.diff(arr[0], axis=0)
# c = np.diff(arr[0], axis=1)
z = np.zeros((1,512))
b = np.concatenate((b, z), axis=0)
c = np.concatenate((c, z.T), axis=1)
print(b.shape)
print(c.shape)
a = b - c



a_flat = a.flatten()
a0 = np.nanmedian(a) - 3 * np.nanstd(a)
a1 = np.nanmedian(a) + 3 * np.nanstd(a)
# plt.title(f'{np.min(a_flat)} --> {np.max(a_flat)}')
# plt.hist(a_flat, bins=1000)
# plt.axvline(a0, c='k')
# plt.axvline(a1, c='k')
# plt.yscale('log')
# plt.show()


out = np.array(arr, copy=True) 
for i in trange(0, a.shape[0]-2):
    for j in range(0, a.shape[1]-2):
        if a[i+1,j] > a1 and a[i,j+1] < a0:
            out[i+1,j+1] = np.median(out[i:i+2, j:j+2]) #np.NaN

plot_image(out, 'Hot pixels?')

# plot_image(a, 'Diff wrt axes 0 & 1', vmin=a0, vmax=a1)
plot_image(arr, 'arr min, max', vmin=np.nanmin(arr), vmax=np.nanmax(arr))
# # plot_image(arr[0], 'arr[0] min, max', vmin=np.nanmin(arr[0]), vmax=np.nanmax(arr[0]))
# # plot_image(arr[0], 'arr[0] +/- 3 sigma', vmin=np.nanmedian(arr[0])-3*np.nanstd(arr[0]), vmax=np.nanmedian(arr[0])+3*np.nanstd(arr[0]))
# # plot_image(arr[-1], 'arr[-1] min, max', vmin=np.nanmin(arr[-1]), vmax=np.nanmax(arr[-1]))


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [5]:
# Std. dev. over projections
print(arr_3d.shape)
# arr_px = arr_3d[:,100,100]
std_over_proj = np.std(arr_3d, axis=(1,2))
plt.plot(std_over_proj)
plt.show()

(39, 10, 512, 512)


In [6]:
image_viewer = sitk.ImageViewer()
image_viewer.SetApplication("C:\\Software\\Fiji.app\\ImageJ-win64.exe")
# image_viewer.SetApplication(r"C:\Program Files\Mango\Mango.exe")
# image_viewer.SetApplication(r"C:\Program Files\ITK-SNAP 3.8\bin\ITK-SNAP.exe")
image_viewer.SetTitle(proj_file_path.replace('\\','_').replace(':',''))
image_viewer.Execute(sitk.GetImageFromArray(arr))